In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

### <font color='blue'> OECD Data </font>

### Import Data

In [2]:
# to make a list of csvs titles that will be nested together
# dl = download

#to read csv of data

## data source: OECD
dl_avg_annual_hrs_worked_df = pd.read_csv('../data/OECD_Avg_annual_hours_worked_per_worker_original_file.csv')
dl_avg_annual_wages_df = pd.read_csv('../data/OECD_Average_annual_wages_original_file.csv')
dl_causes_of_mortality_df = pd.read_csv('../data/OECD_Causes_of_mortality_original_file.csv')
dl_composite_leading_indicators_df = pd.read_csv('../data/OECD_Compsite_Leading_Indicators_original_file.csv')


### High-level Data Cleaning & Aggregation

In [3]:
# to rename the 'value' column to indicate the content

# list of column headers by df 
value_cols = ['Avg. Work Hours (Annual)','Avg. Wages (Annual)','Mortality Causes','CLI Values (Monthly)']
# dict of dataframes 
oecd_dfs = {0: dl_avg_annual_hrs_worked_df, 
            1: dl_avg_annual_wages_df,
            2: dl_causes_of_mortality_df, 
            3: dl_composite_leading_indicators_df
           }

## lists for editing dataframes
list_of_cols_to_drop = ["TIME",
                        "YEA",
                        "Unit Code",
                        "EMPSTAT",
                        "Frequency",
                        "FREQUENCY",
                        "Measure",
                        "PowerCode Code",
                        "PowerCode",
                        "SERIES",
                        "Reference Period Code",
                        "Reference Period",
                        "Flag Codes",
                        "SUBJECT",
                        "Reference",
                        "VAR",
                        "Flags"]


In [4]:
i = 0

# iterate through dataframes 
while i < len(oecd_dfs):
    # rename columns per dataframe
    ## all edited dataframes are now stored in oecd_dfs object 
    oecd_dfs[i] = oecd_dfs[i].rename(columns= {"COUNTRY":"COU",
                                 "LOCATION":"COU",
                                 "UNIT":"Unit",
                                 "Currency": "Unit",
                                 "Year":"Time",
                                 "Variable":"Description", 
                                 "Subject":"Description",
                                 "Employment status":"Description",
                                 "Series":"Description"
                                 })
#                                  "Value":value_cols[i]})
    
    oecd_dfs[i]["Dataset"] = value_cols[i]
    
    # drop columns per dataframe
    for item in list_of_cols_to_drop: 
        # test if column exists in dataframe
        if item in oecd_dfs[i].columns:
            oecd_dfs[i] = oecd_dfs[i].drop(columns= item)
        else:
            continue

    
    # repostion columns in dataframe
    # get a list of of columns in the dataframe
    print(f"before columns {list(oecd_dfs[i].columns)}")
    
    # assign to df
    oecd_dfs[i] = oecd_dfs[i][["Dataset","COU","Country","Time","Description","Value","Unit"]]
    
    
    print(f"Completed df {i} of 3")
    print(f"final columns {list(oecd_dfs[i].columns)}")
    print("=========================")   
    
    
    i += 1

before columns ['COU', 'Country', 'Description', 'Time', 'Unit', 'Value', 'Dataset']
Completed df 0 of 3
final columns ['Dataset', 'COU', 'Country', 'Time', 'Description', 'Value', 'Unit']
before columns ['COU', 'Country', 'Description', 'Time', 'Unit', 'Value', 'Dataset']
Completed df 1 of 3
final columns ['Dataset', 'COU', 'Country', 'Time', 'Description', 'Value', 'Unit']
before columns ['Description', 'Unit', 'COU', 'Country', 'Time', 'Value', 'Dataset']
Completed df 2 of 3
final columns ['Dataset', 'COU', 'Country', 'Time', 'Description', 'Value', 'Unit']
before columns ['Description', 'COU', 'Country', 'Time', 'Unit', 'Value', 'Dataset']
Completed df 3 of 3
final columns ['Dataset', 'COU', 'Country', 'Time', 'Description', 'Value', 'Unit']


In [5]:
# to concatenate dataframes
oecd_df = pd.concat([oecd_dfs[(0)],oecd_dfs[(1)],oecd_dfs[(2)],oecd_dfs[(3)]],ignore_index=True)
print(f"Combined all df")
print("=========================")

Combined all df


In [6]:
oecd_df = oecd_df.dropna(axis=0, how='any')

In [7]:
oecd_df.to_csv('../resources/OECD_Dataframes.csv', sep=',' , encoding= 'utf-8', index=False)

### Reshaping Data Tables for Machine Learning

In [8]:
## show that the same amount of countries in the description .value_counts(sort=True)
oecd_dfs[0].loc[:,['Description',"COU"]].groupby("Description")['COU'].nunique().sort_values()

Description
Dependent employment    32
Total employment        39
Name: COU, dtype: int64

In [9]:
i = 0

actual_descriptions = ['Total employment','In 2018 constant prices at 2018 USD PPPs', 'All causes of death','Amplitude adjusted (CLI)']

while i < len(oecd_dfs):
    # show that the same amount of countries in the description
    results =  oecd_dfs[i].loc[:,['Description',"COU"]].groupby("Description")['COU'].nunique().sort_values(ascending=False)
    actual = results.filter(items = [actual_descriptions[i]])
    
    print('=========')
    print("Actual Description Used:")
    print(actual)
    print('-------------------')
    print("Unqiue COU by Description:")
    print(results)
    
    i+= 1

Actual Description Used:
Description
Total employment    39
Name: COU, dtype: int64
-------------------
Unqiue COU by Description:
Description
Total employment        39
Dependent employment    32
Name: COU, dtype: int64
Actual Description Used:
Description
In 2018 constant prices at 2018 USD PPPs    35
Name: COU, dtype: int64
-------------------
Unqiue COU by Description:
Description
In 2018 constant prices at 2018 USD PPPs    35
Current prices in NCU                       35
2018 constant prices and NCU                35
Name: COU, dtype: int64
Actual Description Used:
Description
All causes of death    41
Name: COU, dtype: int64
-------------------
Unqiue COU by Description:
Description
Tuberculosis                                                    41
Diseases of the blood and blood-forming organs                  41
External causes of mortality                                    41
Transport Accidents                                             41
Diseases of the skin and subcutan

### Consolidating the OECD tables into a Pivot Table

In [10]:
# to make a list of years with a numeric data type for filtering tables
years = [*range(2010,2018,1)]
years

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

In [11]:
descriptions = ['Total employment','In 2018 constant prices at 2018 USD PPPs','All causes of death',]

In [12]:
len(oecd_dfs)-2
oecd_dfs[2].head()

,Dataset,COU,Country,Time,Description,Value,Unit
0,Mortality Causes,AUS,Australia,2000,All causes of death,815.7,TXCMILTX
1,Mortality Causes,AUS,Australia,2001,All causes of death,787.9,TXCMILTX
2,Mortality Causes,AUS,Australia,2002,All causes of death,791.2,TXCMILTX
3,Mortality Causes,AUS,Australia,2003,All causes of death,762.2,TXCMILTX
4,Mortality Causes,AUS,Australia,2004,All causes of death,753.4,TXCMILTX


#### To Consolidate Pivot Tables

In [13]:
OECD_pivot_tables=[]

## CLI not included because it is an annual report

for dfs in range(len(oecd_dfs)-1):
    for year in range(len(years)):
        
        # to filter dataframes by description value
        df = oecd_dfs[dfs].loc[(oecd_dfs[dfs]['Time'] == years[year]) & (oecd_dfs[dfs]['Description'] == descriptions[dfs])]
        
        
        # to reshape the data
        oecd = pd.pivot_table(df, values='Value', index=['COU','Country'], columns=['Dataset'], aggfunc= np.sum)
    
        # data cleaning
        oecd_tables = oecd.dropna(axis=0).reset_index()
    
        # append to list
        OECD_pivot_tables.append(oecd_tables)
    
        print(f"Completed {dfs} Dataframe for year: {years[year]}")

Completed 0 Dataframe for year: 2010
Completed 0 Dataframe for year: 2011
Completed 0 Dataframe for year: 2012
Completed 0 Dataframe for year: 2013
Completed 0 Dataframe for year: 2014
Completed 0 Dataframe for year: 2015
Completed 0 Dataframe for year: 2016
Completed 0 Dataframe for year: 2017
Completed 1 Dataframe for year: 2010
Completed 1 Dataframe for year: 2011
Completed 1 Dataframe for year: 2012
Completed 1 Dataframe for year: 2013
Completed 1 Dataframe for year: 2014
Completed 1 Dataframe for year: 2015
Completed 1 Dataframe for year: 2016
Completed 1 Dataframe for year: 2017
Completed 2 Dataframe for year: 2010
Completed 2 Dataframe for year: 2011
Completed 2 Dataframe for year: 2012
Completed 2 Dataframe for year: 2013
Completed 2 Dataframe for year: 2014
Completed 2 Dataframe for year: 2015
Completed 2 Dataframe for year: 2016
Completed 2 Dataframe for year: 2017


In [14]:
# to review pivot tables
OECD_pivot_tables[23].head(10)

Dataset,COU,Country,Mortality Causes
0,AUT,Austria,171632.6
1,CZE,Czech Republic,228890.0
2,HUN,Hungary,270917.7
3,ISL,Iceland,8498.2
4,LTU,Lithuania,88016.7


In [15]:
# to slice list of pivot tables by year

dataframes = [*range(len(OECD_pivot_tables))]
print(dataframes)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [16]:
def slice_per(source, step):
    return [source[i::step] for i in range(step)]

yearly_df = slice_per(dataframes,8)
yearly_df

[[0, 8, 16],
 [1, 9, 17],
 [2, 10, 18],
 [3, 11, 19],
 [4, 12, 20],
 [5, 13, 21],
 [6, 14, 22],
 [7, 15, 23]]

#### Join Pivot Tables for Machine Learning

In [17]:
# to inner join the pivot tables per the indexes
oecd_ml_df = []


try:
    for index in yearly_df:            
        # to merge 3 pivot tables per dataset column
        oecd_df_part1 = pd.merge(OECD_pivot_tables[index[0]], OECD_pivot_tables[index[(0+1)]], on=['COU','Country'], how='inner')

        # to merge 3 pivot tables per dataset column
        oecd_df_part2 = pd.merge(oecd_df_part1, OECD_pivot_tables[index[(0+2)]], on=['COU','Country'], how='inner')

        print(f"Completed merge of pivot table {index[0]} & {(index[0+1])} & {(index[0+2])}")

        oecd_ml_df.append(oecd_df_part2)
        print(f"Pivot Table created of indexes:{index}")
                   
except IndexError:
    pass

Completed merge of pivot table 0 & 8 & 16
Pivot Table created of indexes:[0, 8, 16]
Completed merge of pivot table 1 & 9 & 17
Pivot Table created of indexes:[1, 9, 17]
Completed merge of pivot table 2 & 10 & 18
Pivot Table created of indexes:[2, 10, 18]
Completed merge of pivot table 3 & 11 & 19
Pivot Table created of indexes:[3, 11, 19]
Completed merge of pivot table 4 & 12 & 20
Pivot Table created of indexes:[4, 12, 20]
Completed merge of pivot table 5 & 13 & 21
Pivot Table created of indexes:[5, 13, 21]
Completed merge of pivot table 6 & 14 & 22
Pivot Table created of indexes:[6, 14, 22]
Completed merge of pivot table 7 & 15 & 23
Pivot Table created of indexes:[7, 15, 23]


In [18]:
oecd_ml_df[4].tail()

Dataset,COU,Country,Avg. Work Hours (Annual),Avg. Wages (Annual),Mortality Causes
30,PRT,Portugal,1714.0,25360.07583,215827.8
31,SVK,Slovak Republic,1760.0,22510.86712,108798.6
32,SVN,Slovenia,1681.9,34282.48437,42895.9
33,SWE,Sweden,1470.0,42444.08578,183045.0
34,USA,United States,1784.0,60617.97202,5257849.6


### <font color = 'green'> World Bank Data </font>

#### Importing Data

In [ ]:
#to read csv of data
df= pd.read_csv('../../../WDIData.csv')

#to read csv of country names
df_country_names= pd.read_csv('../data/WDI_Country_Code_and_Names.csv',encoding = "ISO-8859-1")


#to read csv of indicators
df_list_indicators = pd.read_csv('../data/WDI_list_of_reviewed_indicators.csv')

#### Data Cleaning

In [ ]:
#to grab the headers of the dataset
df_headers=list(df.columns.values)

#only pull the headers that are years
df_years = df_headers[-59:]

#only select previous years before 2010
df_years_drop = df_years[0:50]
#to make column '2018' a list
df_years_drop_2018 = list([df_years[-1]])

# to combine lists of years into 1 drop line
df_drop = df_years_drop + df_years_drop_2018

#to drop the years and create a summarized df
df_columns_removed = df.drop(df_drop,axis=1)
df_columns_removed.head()

Assigning Country Code

In [ ]:
#to preview df_country_names
df_country_names.head()

#make a list of country short name
country_short_names = list(df_country_names['Short Name'])

#filter df by short names list
df_filter_cols_nd_cols_rem = df_columns_removed.loc[df_columns_removed['Country Name'].isin(country_short_names)]

In [ ]:
df_filter_cols_nd_cols_rem.head()

Filter indicators

In [ ]:
df_list_indicators.head()

In [ ]:
indicators =  list(df_list_indicators['Indicator Name'])
print(indicators)

#filter by indicators
df_countries_indiciator = df_filter_cols_nd_cols_rem.loc[df_filter_cols_nd_cols_rem['Indicator Name'].isin(indicators)]

#remove unnessecary columns
df_wdi = df_countries_indiciator.drop(columns='Indicator Code')

#### Reshaping Data

In [ ]:
df_wdi.head()

In [ ]:
years = list(df_wdi.iloc[:,3:].columns)
years

In [ ]:
i = 0
world_bank_indicators = []


while i < len(years):
    
    # to create a df of series
    wdi_data = df_wdi[['Country Code','Country Name', 'Indicator Name', years[i]]]
    wdi_data = wdi_data.rename(columns={'Country Code':'COU','Country Name':'Country'})
    
    
    # to reshape the data
    wdi = pd.pivot_table(wdi_data, values=years[i], index=['COU','Country'], columns=['Indicator Name'], aggfunc= np.sum)
    wdi.reset_index()
    
    # data cleaning
    wdi_df = wdi.dropna(axis=0, how='all').reset_index()
    
    #append a country code
    
    
    # append to list
    world_bank_indicators.append(wdi_df)
    
    # to create csv
    world_bank_indicators[i].to_csv(f'../../../WDI_Data_Happiness_Table_{years[i]}.csv', sep=',' , encoding= 'utf-8', index=False)
    
    print(f"Complete Dataframe for year: {years[i]}")

    i += 1

In [ ]:
world_bank_indicators[0].head()

### <font color='red'> To Combine both datasets </font>

In [ ]:
print(len(world_bank_indicators))
print(len(oecd_ml_df))

## Both datasets are the same length, thus we can merge them to each other 

OECD data has less rows, thus we will perform an inner merge between the World Bank Data and the OECD data.

In [ ]:
i = 0 

ml_datasets = []

while i < len(oecd_ml_df):
    
    # to merge 3 pivot tables per dataset column
    ml_df = pd.merge(oecd_ml_df[i], world_bank_indicators[i], on=['COU','Country'], how='inner')
    ml_df['Year'] = years[i]
    ml_datasets.append(ml_df)
    
    # to create csv
    ml_datasets[i].to_csv(f'../resources/Machine_Learning_Dataset_{years[i]}.csv', sep=',' , encoding= 'utf-8', index=False)
    
    print(f'Completed dataset {i} of 7')
    
    i +=1
    

In [ ]:
ml_datasets[0].head()

### <font color='magenta'> UN Happiness Report </font>

In [ ]:
#to read csv un happiness data names
un_happiness_report = pd.read_csv('../data/world_happiness_report_2019_original_file.csv',encoding = "ISO-8859-1")

In [ ]:
# years is from earlier in the report
years

In [ ]:
un_happiness_report.head()

In [ ]:
# to filter by year to fit other datasets 2010 to 2017
un_happiness_report = un_happiness_report.loc[un_happiness_report['Year'] == 2010]
happiness_merge_ready = un_happiness_report.drop(columns=['Year'])
merge_ready = happiness_merge_ready.rename(columns={'Country name':'Country'})
merge_ready.head()

In [ ]:
years_to_review = [*range(2010,2018)]
years_to_review

In [ ]:
i = 0

happiness_datasets = []

while i < len(years):
    
    # to filter by year to fit other datasets 2010 to 2017
    un_happiness_report = un_happiness_report.loc[un_happiness_report['Year'] == years_to_review[i]]
    happiness_merge_ready = un_happiness_report.drop(columns=['Year'])
    merge_ready = happiness_merge_ready.rename(columns={'Country name':'Country'})
    
    
    # to merge 3 pivot tables per dataset column
    life_ml_df = pd.merge(ml_datasets[i], merge_ready, on=['COU','Country'], how='inner')
    
    happiness_datasets.append(life_ml_df)
    
    # to create csv
    happiness_datasets[i].to_csv(f'../resources/Machine_Learning_Dataset_{years[i]}.csv', sep=',' , encoding= 'utf-8', index=False)
    
    
    print(f"Merged UN Year {years[i]} with OECD & WDI Data")
    i+=1
    

In [ ]:
happiness_datasets[0]